### Library

In [ ]:
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
data_dir = './Data'

### Data processing
